In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from imp import reload
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd

from tensorflow.keras import datasets, layers, models

import textwrap
import random
import json

from simpletransformers.classification import ClassificationModel
import pandas as pd

In [ ]:
data_path = '../noncomm_use_subset/pmc_json/'

In [ ]:
# Load up files from a directory

In [ ]:
# Create a class object to represent papers/articles with contained methods